## 학습 단계

1. 라이브러리 가져오기(torch, torch.nn, torchvision, matplotlib 등)
2. GPU 사용 설정, random value를 위한 seed 설정
3. 학습에 사용되는 parameter (learning_rate, training_peochs, batch_size, etc)
4. 데이터셋 가저오고 학습에 편리하게 loader 만들기
5. 학습 모델 만들기 (class CNN 같은 것)
6. Loss function(Criterion)을 선택, 최적화 도구도 선택
7. 모델 학습 및 loss check(criterion의 output)
8. 학습된 모델의 성능 확인

In [1]:
# 1 라이브러리 가져오기
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
# 2 GPU 사용 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# 3 parameters
learning_rate = 0.001
epochs = 15
batch_size = 100

In [4]:
# 4 MNIST dataset 가져오기

mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform = transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST-data/',
                         train=False,
                         transform = transforms.ToTensor(),
                         download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 241089894.66it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 37399102.76it/s]


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 77598530.10it/s]


...

Extracting MNIST-data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST-data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 74245471.20it/s]

Extracting MNIST-data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST-data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 6526388.75it/s]

Extracting MNIST-data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST-data/MNIST/raw



In [5]:
# 데이터 로더 만들기
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)

In [6]:
# 5 모델 만들기
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__() # 매우 중요, 이거 빼먹으면 안됨

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels = 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # fully connected layer가 필요
        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias=True) # layer2까지 진행할 시, view로 펼치면 7*7*64, 정답은 0~9의 숫자
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    # 신경망 진행 명령
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)

        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [7]:
model = CNN().to(device)

In [8]:
value = torch.Tensor(1,1,28,28).to(device)
print( model(value).shape)

torch.Size([1, 10])


In [9]:
# 6 loss function 만들기
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

In [10]:
# 7 훈련 함수
total_batch = len(data_loader)
for epoch in range(epochs):
    avg_cost = 0

    for x, y in data_loader:
        x = x.to(device)
        y = y.to(device)

        hypothesis = model(x)
        cost = criterion(hypothesis, y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    print('Epoch: {:4d}, Cost: {:.6f}'.format(epoch, avg_cost))

Epoch:    0, Cost: 0.159917
Epoch:    1, Cost: 0.043036
Epoch:    2, Cost: 0.030721
Epoch:    3, Cost: 0.020789
Epoch:    4, Cost: 0.017655
Epoch:    5, Cost: 0.014977
Epoch:    6, Cost: 0.014174
Epoch:    7, Cost: 0.010090
Epoch:    8, Cost: 0.008991
Epoch:    9, Cost: 0.008423
Epoch:   10, Cost: 0.009233
Epoch:   11, Cost: 0.005725
Epoch:   12, Cost: 0.006440
Epoch:   13, Cost: 0.006094
Epoch:   14, Cost: 0.005372


In [14]:
with torch.no_grad():
    x_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    y_test = mnist_test.test_labels.to(device)

    prediction = model(x_test)
    correct_prediction = torch.argmax(prediction, 1) == y_test
    accuracy = correct_prediction.float().mean()

    print('Accuracy: {:.6f}'.format(accuracy))

Accuracy: 0.987900


In [ ]:
# 부록: layer 크기 계산하는 법
input = torch.Tensor(1,1,28,28)

conv1 = nn.Conv2d(1, 32, 3, stride=1, padding=1)
pool = nn.MaxPool2d(2)
conv2 = nn.Conv2d(32, 64, 3, stride=1, padding=1)

out1 = conv1(input)
out2 = pool(out1)
out3 = conv2(out2)
out4 = pool(out3)

print(out1.shape)
print(out2.shape)
print(out3.shape)
print(out4.shape)

torch.Size([1, 32, 28, 28])
torch.Size([1, 32, 14, 14])
torch.Size([1, 64, 14, 14])
torch.Size([1, 64, 7, 7])
